<a href="https://colab.research.google.com/github/Dezvi/HeatMap2D/blob/main/HeatMap2D.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>



> We install the required packages not included in colab and import the required packages.


In [ ]:
%matplotlib inline
!pip install pycuda
#Import packages we need
import numpy as np
from matplotlib import animation, rc, cm
from matplotlib import pyplot as plt

import pycuda
import pycuda.compiler as cuda_compiler
import pycuda.driver as cuda_driver
from pycuda.gpuarray import GPUArray

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 10.6 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 70.6/70.6 kB 10.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.7/78.7 kB 11.5 MB/s eta 0:00:00
  Created wheel for pycuda: filename=pycuda-2024.1-cp310-cp310-linux_x86_64.whl size=661205 sha256=4798630e581858853203e0ef9c3be781442fce4133ee5f117f26cff82b7ea799
  Stored in directory: /root/.cache/pip/wheels/12/34/d2/9a349255a4eca3a486d82c79d21e138ce2ccd90f414d9d72b8
Successfully built pycuda


We create the class and the kernel that will do the step by step heatmap simulation


In [ ]:
class HeatEquation2DGPU(object):
    def __init__(self):
        pass

    def initialize(self, u0, kappa, dx, dt, dy, block_width=8, block_height=8):
        self.kappa = np.float32(kappa)
        self.dx = np.float32(dx)
        self.dt = np.float32(dt)
        self.nx = np.int32(u0.shape[1]-2)

        self.dy = np.float32(dy)
        self.ny = np.int32(u0.shape[0]-2)

        self.block_size = (block_width, block_height, 1)
        self.grid_size = (int(np.ceil(self.nx / block_width)), int(np.ceil(self.ny / block_height)), 1)
        self.stream = cuda_driver.Stream()

        self.u1_g = GPUArray(u0.shape, u0.dtype)
        self.u0_g = GPUArray(u0.shape, u0.dtype)

        self.u0_g.set_async(u0, stream=self.stream)


        cuda_kernel = \
        """
        #define BLOCK_WIDTH {:d}
        #define BLOCK_HEIGHT {:d}
        """.format(block_width, block_height) \
        + \
        """
        __global__ void heatEqn(float* u1, const float* u0, float kappa, float dx, float dt, int nx, float dy, int ny) {
            //Plus one to skip the "ghost cells"
            int x_blockdim = blockDim.x;

            int i = blockIdx.x*x_blockdim + threadIdx.x + 1;
            int tx = threadIdx.x + 1;

            int y_blockdim = blockDim.y;

            int j = blockIdx.y*y_blockdim + threadIdx.y + 1;
            int ty = threadIdx.y + 1;


            //First read into shared memory, including the local ghost cells / apron
            __shared__ float u0_shared[BLOCK_HEIGHT+2][BLOCK_WIDTH+2];
            for (int i = threadIdx.y; i < BLOCK_HEIGHT+2; i += y_blockdim) {
                int min_y = min(blockIdx.y * y_blockdim + i, ny);
                for (int j = threadIdx.x; j < BLOCK_WIDTH+2; j += x_blockdim) {
                    int min_x = min(blockIdx.x * x_blockdim + j, nx);
                    u0_shared[i][j] = u0[min_y * (nx + 2) + min_x];
                }
            }
            __syncthreads();

            //Then fix the boundary conditions
            //[0 | 1 2 3 4 | 5]
            if (i == 1) {
                u0_shared[ty][tx-1] = u0_shared[ty][tx];
            }
            else if (i == nx) {
                u0_shared[ty][tx+1] = u0_shared[ty][tx];
            }
            else if (j == 1) {
                u0_shared[ty-1][tx] = u0_shared[ty][tx];
            }
            else if (j == ny) {
                u0_shared[ty+1][tx] = u0_shared[ty][tx];
            }

            if (i >= 1 && i <= nx && j >= 1 && j <= ny) {
                u1[j*(nx+2) + i] = u0_shared[ty][tx] + kappa*dt/(dx*dx) * (u0_shared[ty][tx-1] - 2.0f*u0_shared[ty][tx] + u0_shared[ty][tx+1])
                    + kappa*dt/(dy*dy) * (u0_shared[ty-1][tx] - 2.0f*u0_shared[ty][tx] + u0_shared[ty+1][tx]);
            }
        }
        """
        self.module = cuda_compiler.SourceModule(cuda_kernel, \
                                            options=['--use_fast_math'])
        self.heat_eqn_kernel = self.module.get_function("heatEqn");
        self.heat_eqn_kernel.prepare("PPfffifi")



    def step(self):
        self.heat_eqn_kernel.prepared_async_call(self.grid_size, self.block_size, self.stream, \
                                                    self.u1_g.gpudata, self.u0_g.gpudata, \
                                                    self.kappa, self.dx, self.dt, self.nx, self.dy, self.ny)
        self.u0_g, self.u1_g = self.u1_g, self.u0_g


    def download(self):
        #self.u2 = cuda_driver.pagelocked_empty(u1.shape, u1.dtype)
        u0 = np.empty(self.u0_g.shape, self.u0_g.dtype)
        self.u0_g.get(u0)
        return u0


#Create test input data
kappa = 1.0
nx = 100
dx = 1.0

ny = 50
dy = 2.0

dt = 0.4*min(dx**2 / (2.0*kappa), dy**2 / (2.0*kappa))


u0 = np.zeros((ny, nx), dtype=np.float32)
for j in range(ny):
    for i in range(nx):
        x = (i - nx/2.0) * dx
        y = (j - ny/2.0) * dy
        if (np.sqrt(x**2 + y**2) < 10*min(dx, dy)):
            u0[j, i] = 10.0

import pycuda.autoinit

pycuda.tools.make_default_context()
simulator = HeatEquation2DGPU()
simulator.initialize(u0, kappa, dx, dt, dy)


I have added dy, ny and block height on the kernel, then I have tried to do the same operations that were happening for x, for the y dimension.

Instead of linspace I use plt imshow with 2 dimensions.

In [ ]:
fig = plt.figure(figsize=(12, 8))
ax = fig.add_subplot(111)

heat2d = plt.imshow(u0, extent=[0, nx*dx, 0, ny*dy])
plt.colorbar()

def animate(i):
    print(".", end='', flush=True)
    if (i > 0):
        for k in range(10):
            simulator.step()

    u1 = simulator.download()

    heat2d.set_data(u1)
    print(np.sum(u1[1:-1]))

plt.rcParams["animation.html"] = "html5"
anim = animation.FuncAnimation(fig, animate, range(50), interval=100)
plt.close()
anim

.1470.0
.1470.0
.1470.0
.1470.0
.1470.0
.1469.9999
.1469.9999
.1469.9999
.1469.9998
.1469.9999
.1469.9998
.1469.9999
.1469.9998
.1470.0
.1469.9998
.1469.9998
.1469.9998
.1469.9998
.1469.9998
.1469.9998
.1469.9998
.1469.9998
.1469.9998
.1469.9996
.1469.9998
.1469.9996
.1469.9998
.1469.9998
.1469.9996
.1469.9995
.1469.9998
.1469.9995
.1469.9996
.1469.9996
.1469.9998
.1469.9995
.1469.9995
.1469.9995
.1469.9995
.1469.9995
.1469.9995
.1469.9995
.1469.9995
.1469.9995
.1469.9994
.1469.9993
.1469.9993
.1469.9993
.1469.9993
.1469.999
.1469.999
